<a href="https://colab.research.google.com/github/HasunJung/llm_langchain/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

chat bot

In [186]:
#최초 실행이라면
!pip install -q langchain langchain-google-genai pandas kor langchain-text-splitters sentence-transformers chromadb

import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# ChatGoogleGenerativeAI 언어 모델을 "gemini-pro" 모델로 초기화합니다. 답변의 일관성을 위해 temperature는 0으로 설정
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.

# Document Loaders

In [146]:
import ssl
import urllib.request
from IPython.display import Markdown
import json

url = 'https://lineagem.plaync.com/api/v2/lineagem/guide/%EC%B0%BD%EA%B3%A0%20%EC%8B%9C%EC%8A%A4%ED%85%9C'

# Set up SSL context to allow legacy TLS versions
ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT

response = urllib.request.urlopen(url, context=ctx)
guide_data = response.read().decode("utf-8")

guide_json = json.loads(guide_data)



In [171]:
import pandas as pd

df = pd.DataFrame({
    'name' : guide_json['name'],
    'summary' : guide_json['summary'],
    'content' : guide_json['content'],
    'paragraphList' : guide_json['paragraphList'],
    'test' : guide_json['paragraphList']['0']['content']
}, index=[0])



#print(len(guide_json['paragraphList']))

from langchain_community.document_loaders import DataFrameLoader


loader = DataFrameLoader(df, page_content_column = 'test')
docs = loader.load()

print(docs[0].page_content)


<p>창고는 소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간으로 창고에 있는 아이템은 계정 내 캐릭터 모두 공유할 수 있습니다.</p><p>&lt;리니지M&gt;에서는 캐릭터의 STR과 CON, 아이템에 붙어있는 무게 게이지에 따라 인벤토리에 넣을 수 있는 소지품의 한계치(최대 무게)가 결정됩니다.</p><p>무게 게이지가 50% 이상이 되면 다양한 패널티를 받기 때문에 불필요한 아이템은 상점에 팔거나 창고에 보관해서 무게 게이지를 관리해야 합니다.</p><p><strong>&lt;무게 게이지에 따른 패널티&gt;</strong></p><p>* 50% 이상 : HP 및 MP 자동 회복 불가</p><p>* 83% 이상 : 공격 및 마법 사용 불가</p><p><br></p>


# RecursiveCharacterTextSplitter

In [182]:
from kor.documents.html import MarkdownifyHTMLProcessor
md = MarkdownifyHTMLProcessor().process(docs[0])


from langchain_text_splitters import RecursiveCharacterTextSplitter
split_docs = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=0).split_documents([md])

print(split_docs[0].page_content)


창고는 소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간으로 창고에 있는 아이템은 계정 내 캐릭터 모두 공유할 수 있습니다.

<리니지M>에서는 캐릭터의 STR과 CON, 아이템에 붙어있는 무게 게이지에 따라 인벤토리에 넣을 수 있는 소지품의 한계치(최대 무게)가 결정됩니다.

무게 게이지가 50% 이상이 되면 다양한 패널티를 받기 때문에 불필요한 아이템은 상점에 팔거나 창고에 보관해서 무게 게이지를 관리해야 합니다.

**<무게 게이지에 따른 패널티>**

\* 50% 이상 : HP 및 MP 자동 회복 불가

\* 83% 이상 : 공격 및 마법 사용 불가


In [187]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


# VectorDB
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=split_docs, embedding=ko_embedding)

In [191]:
similar_docs = vectordb.similarity_search("창고 무게 게이지에 따른 패널티를 설명해줘")
print(similar_docs[0].page_content)

창고는 소지품에 있는 아이템을 맡기거나 찾을 수 있는 공간으로 창고에 있는 아이템은 계정 내 캐릭터 모두 공유할 수 있습니다.

<리니지M>에서는 캐릭터의 STR과 CON, 아이템에 붙어있는 무게 게이지에 따라 인벤토리에 넣을 수 있는 소지품의 한계치(최대 무게)가 결정됩니다.

무게 게이지가 50% 이상이 되면 다양한 패널티를 받기 때문에 불필요한 아이템은 상점에 팔거나 창고에 보관해서 무게 게이지를 관리해야 합니다.

**<무게 게이지에 따른 패널티>**

\* 50% 이상 : HP 및 MP 자동 회복 불가

\* 83% 이상 : 공격 및 마법 사용 불가
